In [7]:
require(comprehenr)
require(glue)
require(tidyverse)
require(bipartite)
require(stringr)
require(parallel)
require(pbmcapply)
require(vegan)
source("../../code/feature_computation/utils.R")

In [9]:
method = "patefield" # options:
                     # patefield - created weighted networks
                     # bascompte - created binary networks
                     # vasquez - created both types of nwetworks
                     # swap_web - creates both types of networks
networks_dir = "../../data/networks/all/"
networks_types = c("binary", "binarized_weighted") # "weighted") #
null_networks_dir = glue("../../data/networks/null_", method, "/")
networks_data_path = "../../data/networks/networks_metadata.csv"

In [10]:
dir.create(null_networks_dir)
for (net_type in networks_types)
    {
        dir.create(paste(null_networks_dir, net_type, sep=""))
    }

Warning message in dir.create(null_networks_dir):
“'../../data/networks/null_patefield' already exists”
Warning message in dir.create(paste(null_networks_dir, net_type, sep = "")):
“'../../data/networks/null_patefield/binary' already exists”
Warning message in dir.create(paste(null_networks_dir, net_type, sep = "")):
“'../../data/networks/null_patefield/binarized_weighted' already exists”


In [11]:
networks_data = read.csv(networks_data_path)
networks_data = networks_data[networks_data$included_in_analysis == "True", ]
networks_data = networks_data[networks_data$network_type %in% networks_types, ]
full_network_paths = as.vector(networks_data$processed_path)
length(full_network_paths)

[1] 419

In [12]:
simulate_null_networks <- function(path, nsim=100, method="patefield")
{
    network = process_network(path) 
    # is_weighted = any(network != 1 & network != 0)
    if (dim(network)[1] < 2)
        {
        return (0)
        }
    if (dim(network)[2] < 2)
        {
        return (0)
        } 
    output_dir = str_replace(str_replace(path, networks_dir, null_networks_dir), ".csv", "/")
    dir.create(output_dir, showWarnings = FALSE)
    num_files <- length(list.files(output_dir))
    if (num_files == nsim)
    {
        return(0)
    }
    if (method == "vasquez")
    {
        null_networks = nullmodel(web=network, N=nsim, method="vaznull")
    } else {
        if (method == "patefield")
        {
            null_networks = nullmodel(web=network, N=nsim, method="r2dtable")
        } else {
            if (method == "bascompte")
            {
                null_networks = generare_null_networks(web=network, nsimul=nsim)
            } else {
                null_networks = nullmodel(web=network, N=nsim, method="swap.web")
            }
        }
    }

    print(glue(length(null_networks), " networks were simulated for network ", path))
    for (i in 1:length(null_networks))
    {
        output_path = paste(output_dir, i, ".csv", sep="")
        null_network = data.frame(null_networks[i])
        colnames(null_network) = colnames(network)
        rownames(null_network) = rownames(network)
        write.csv(null_network, output_path)
    }   
    return(0)
}

In [13]:
i = 0
for (path in full_network_paths) {
    print(glue("generating null networks for ", path))
    output_dir = str_replace(str_replace(path, networks_dir, null_networks_dir), ".csv", "/")
    if ((!dir.exists(output_dir)) || (length(list.files(output_dir)) < 100)) {
        res = simulate_null_networks(path, method=method)
        i = i + 1
    }
}

generating null networks for ../../data/networks/all/binary/47.csv
generating null networks for ../../data/networks/all/binary/54.csv
generating null networks for ../../data/networks/all/binary/55.csv
generating null networks for ../../data/networks/all/binary/61.csv
100 networks were simulated for network ../../data/networks/all/binary/61.csv
generating null networks for ../../data/networks/all/binary/69.csv
generating null networks for ../../data/networks/all/binary/182.csv
generating null networks for ../../data/networks/all/binary/183.csv
generating null networks for ../../data/networks/all/binary/184.csv
generating null networks for ../../data/networks/all/binary/186.csv
generating null networks for ../../data/networks/all/binary/187.csv
generating null networks for ../../data/networks/all/binary/188.csv
generating null networks for ../../data/networks/all/binary/189.csv
generating null networks for ../../data/networks/all/binary/190.csv
generating null networks for ../../data/net